In [1]:
import pandas as pd
import re

In [7]:
df = pd.read_csv('immobiliers.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1643 entries, 0 to 1642
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Type de transaction       1643 non-null   object 
 1   Superficie                1643 non-null   object 
 2   Type de bien              1643 non-null   object 
 3   Configuration des pièces  559 non-null    object 
 4   Prix                      1429 non-null   float64
 5   Localisation              1643 non-null   object 
 6   Chambres                  1643 non-null   object 
 7   Annonce                   1643 non-null   object 
 8   Salles de bains           1643 non-null   object 
dtypes: float64(1), object(8)
memory usage: 115.7+ KB


In [4]:
df.tail()

,Type de transaction,Superficie,Type de bien,Configuration des pièces,Prix,Localisation,Chambres,Annonce,Salles de bains
1638,À Vendre,False,Appartement,S+2,286269.0,Sfax,False,Appartement (C2-8) en S+2 au deuxième étage,False
1639,À Vendre,125.8,Appartement,S+2,232730.0,Sfax,2,Appartement (C2-9) en S+2 au deuxième étage,1
1640,À Vendre,183.21,Appartement,S+3,338939.0,Sfax,3,Appartement (A4-9) en S+3 au quatrième étage,3
1641,À Vendre,202.79,Appartement,S+3,375162.0,Sfax,3,Appartement (C2-3) en S+3 au deuxième étage,3
1642,À Vendre,191.02,Appartement,S+3,353387.0,Sfax,3,Appartement (C2-6) en S+3 au deuxième étage,3


In [8]:
# on s'interesse que sur les biens à vendre
# execute right after reading the csv, marra wahda barka
df = df[df['Type de transaction'] == 'À Vendre']
df = df.drop(columns=['Type de transaction'])
df.tail()

,Superficie,Type de bien,Configuration des pièces,Prix,Localisation,Chambres,Annonce,Salles de bains
1638,False,Appartement,S+2,286269.0,Sfax,False,Appartement (C2-8) en S+2 au deuxième étage,False
1639,125.8,Appartement,S+2,232730.0,Sfax,2,Appartement (C2-9) en S+2 au deuxième étage,1
1640,183.21,Appartement,S+3,338939.0,Sfax,3,Appartement (A4-9) en S+3 au quatrième étage,3
1641,202.79,Appartement,S+3,375162.0,Sfax,3,Appartement (C2-3) en S+3 au deuxième étage,3
1642,191.02,Appartement,S+3,353387.0,Sfax,3,Appartement (C2-6) en S+3 au deuxième étage,3


In [ ]:
condition1 = df[(df['Configuration des pièces'].notna()) & (df['Chambres']=='False')]
# print(condition1.shape)
# 12 row où nb chambres n'est affiché que dans l'annonce

# Extraire le nombre de pièces de 'Configuration des pièces' et l'attribuer à 'Chambres'
for idx, row in condition1.iterrows():
    config_pieces = row['Configuration des pièces']
    match = re.search(r'S\+(\d+)', config_pieces)
    if match:
        df.at[idx, 'Chambres'] = int(match.group(1))

df = df.drop(columns=['Configuration des pièces']) # on n'en a plus besoin
df.tail()

,Superficie,Type de bien,Prix,Localisation,Chambres,Annonce,Salles de bains
1638,False,Appartement,286269.0,Sfax,2,Appartement (C2-8) en S+2 au deuxième étage,False
1639,125.8,Appartement,232730.0,Sfax,2,Appartement (C2-9) en S+2 au deuxième étage,1
1640,183.21,Appartement,338939.0,Sfax,3,Appartement (A4-9) en S+3 au quatrième étage,3
1641,202.79,Appartement,375162.0,Sfax,3,Appartement (C2-3) en S+3 au deuxième étage,3
1642,191.02,Appartement,353387.0,Sfax,3,Appartement (C2-6) en S+3 au deuxième étage,3


In [23]:
df = df.drop_duplicates()

In [27]:
# Remplacer toutes les valeurs 'False' dans la colonne "Salles de bains" par la valeur par défaut 1
df['Salles de bains'].replace('False', 1, inplace=True)
df.tail()

,Superficie,Type de bien,Prix,Localisation,Chambres,Annonce,Salles de bains
1638,False,Appartement,286269.0,Sfax,2,Appartement (C2-8) en S+2 au deuxième étage,1
1639,125.8,Appartement,232730.0,Sfax,2,Appartement (C2-9) en S+2 au deuxième étage,1
1640,183.21,Appartement,338939.0,Sfax,3,Appartement (A4-9) en S+3 au quatrième étage,3
1641,202.79,Appartement,375162.0,Sfax,3,Appartement (C2-3) en S+3 au deuxième étage,3
1642,191.02,Appartement,353387.0,Sfax,3,Appartement (C2-6) en S+3 au deuxième étage,3


In [28]:
df['Prix'] = pd.to_numeric(df['Prix'], errors='coerce') # convertir en numérique

In [30]:
# imputer les vm en utilisant la médiane des biens similaires (pour éviter les outliers)
# df['Prix'] = df.groupby(['Localisation', 'Chambres'])['Prix'].transform(lambda x: x.fillna(x.median())) 